# Input Example

In [1]:
!pip install datasets --quiet
!pip install transformers --quiet
!pip install torch --quiet

In [2]:
from datasets import load_dataset

from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification

import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW

torch.set_printoptions(linewidth=1000000)

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load Data

In [3]:
dataset = load_dataset('csv', data_files={'train': '../data/oasst1_train_cleaned.csv'})

Found cached dataset csv (/home/jovyan/.cache/huggingface/datasets/csv/default-03ec878f41594adc/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
100%|██████████| 1/1 [00:00<00:00, 364.53it/s]


In [4]:
dataset = dataset.filter(lambda example: example["message_id"] == 'eae6f970-0971-4422-b701-79e273b2ad55')

Loading cached processed dataset at /home/jovyan/.cache/huggingface/datasets/csv/default-03ec878f41594adc/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-80ae91a355377083.arrow


### Prepare Data

In [5]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

In [6]:
def transform_labels(data):
    humor = data['humor']
    label = 1 if humor > 0.3 else 0
    return {'labels': label}

dataset = dataset.map(transform_labels)

Loading cached processed dataset at /home/jovyan/.cache/huggingface/datasets/csv/default-03ec878f41594adc/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-9d528506d3b044a7.arrow


In [7]:
def tokenize_data(data):
    return tokenizer(data['text'], padding="max_length", truncation=True)

remove_columns = ['message_id', 'role', 'lang', 'humor', 'text']
dataset = dataset.map(tokenize_data, remove_columns=remove_columns, batched=True)

Loading cached processed dataset at /home/jovyan/.cache/huggingface/datasets/csv/default-03ec878f41594adc/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-62b2eb78c0baddfd.arrow


In [8]:
dataset.set_format(type="torch", columns=["input_ids", "token_type_ids", "attention_mask", "labels"])

In [9]:
train_dataset = dataset['train']

### Data Loader

In [10]:
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=1)

### Setup Model

In [11]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

### Example

In [12]:
batch = next(iter(train_dataloader))

In [13]:
print("Example for sentence: ", str(tokenizer.batch_decode(batch['input_ids'])[0])[0:102], "...")

Example for sentence:  [CLS] If you were an FBI agent, would you answer yes to this question? [SEP] [PAD] [PAD] [PAD] [PAD] [ ...


In [14]:
for (key, item) in batch.items():
    if (item.ndim > 1):
        print(key, ": ", str(item[0])[0:100], "...")
    else:
        print(key, ": ", item)

labels :  tensor([1])
input_ids :  tensor([ 101, 1409, 1128, 1127, 1126, 8099, 3677,  117, 1156, 1128, 2590, 4208, 1106, 1142, 2304,  1 ...
token_type_ids :  tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
attention_mask :  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...


In [15]:
model.eval()
with torch.no_grad():
    outputs = model(**batch)

In [16]:
print("Loss: ", outputs.loss)
print("Logits: ", outputs.logits)

Loss:  tensor(0.6226)
Logits:  tensor([[-0.6483, -0.5018]])
